In [1]:
import requests # library for making HTTP requests
import pandas as pd # library for data analysis
import datetime as dt # library for handling date and time objects

In [8]:
# Specify the desired start and end time
start_time = pd.Timestamp(2002, 1, 1)
end_time = pd.Timestamp(2024, 12, 31)

# Specify one or more station IDs or all_stations
stationIds = ['06102']
# Specify one or more parameter IDs or all_parameters
parameterIds = ['temp_mean_past1h']

DMI_URL = 'https://dmigw.govcloud.dk/v2/metObs/collections/observation/items'
api_key = '56642815-d535-418b-bda2-30a8a8ef4999'

# Derive datetime specifier string
datetime_str = start_time.tz_localize('UTC').isoformat() + '/' + end_time.tz_localize('UTC').isoformat()

dfs = []
for station in stationIds:
    for parameter in parameterIds:
        # Specify query parameters
        params = {
            'api-key' : api_key,
            'datetime' : datetime_str,
            'stationId' : station,
            'parameterId' : parameter,
            'limit' : '300000',  # max limit
        }

        # Submit GET request with url and parameters
        r = requests.get(DMI_URL, params=params)
        # Debugging: Print API response status and raw JSON output
        print("Status Code:", r.status_code)  # Check if the request was successful
        print("Response Text:", r.text[:1000])  # Print first 1000 characters of the response
        # Extract JSON object
        json = r.json() # Extract JSON object
        # Convert JSON object to a MultiIndex DataFrame and add to list
        dfi = pd.json_normalize(json['features'])
        if dfi.empty is False:
            dfi['time'] = pd.to_datetime(dfi['properties.observed'])
            # Drop other columns
            dfi = dfi[['time', 'properties.value', 'properties.stationId', 'properties.parameterId']]
            # Rename columns, e.g., 'properties.stationId' becomes 'stationId'
            dfi.columns = [c.replace('properties.', '') for c in dfi.columns]
            # Drop identical rows (considers both value and time stamp)
            dfi = dfi[~dfi.duplicated()]
            dfi = dfi.set_index(['parameterId', 'stationId', 'time'])
            dfi = dfi['value'].unstack(['stationId','parameterId'])
            dfs.append(dfi)

df = pd.concat(dfs, axis='columns').sort_index()
df.head()

Status Code: 200
Response Text: {"type":"FeatureCollection","features":[{"geometry":{"coordinates":[9.7872,55.868],"type":"Point"},"id":"7a042dd2-05b3-7f06-9b53-01967c92a1c8","type":"Feature","properties":{"created":"2024-12-31T00:03:49.049259Z","observed":"2024-12-31T00:00:00Z","parameterId":"temp_mean_past1h","stationId":"06102","value":8.3}},{"geometry":{"coordinates":[9.7872,55.868],"type":"Point"},"id":"e41f92ea-a0f0-2724-6182-ad9b29754c4a","type":"Feature","properties":{"created":"2024-12-30T23:03:29.352688Z","observed":"2024-12-30T23:00:00Z","parameterId":"temp_mean_past1h","stationId":"06102","value":8.7}},{"geometry":{"coordinates":[9.7872,55.868],"type":"Point"},"id":"cacc92ec-af1d-4bc0-14e8-a9804a1b3f6f","type":"Feature","properties":{"created":"2024-12-30T22:03:47.740903Z","observed":"2024-12-30T22:00:00Z","parameterId":"temp_mean_past1h","stationId":"06102","value":8.4}},{"geometry":{"coordinates":[9.7872,55.868],"type":"Point"},"id":"0ec94b6a-2595-5244-e17a-4fdb7856675d",

stationId,06102
parameterId,temp_mean_past1h
time,
2002-01-01 00:00:00+00:00,-3.5
2002-01-01 01:00:00+00:00,-6.6
2002-01-01 02:00:00+00:00,-7.9
2002-01-01 03:00:00+00:00,-8.4
2002-01-01 04:00:00+00:00,-8.6


In [9]:
df.tail()

stationId,06102
parameterId,temp_mean_past1h
time,
2024-12-30 20:00:00+00:00,8.0
2024-12-30 21:00:00+00:00,8.2
2024-12-30 22:00:00+00:00,8.4
2024-12-30 23:00:00+00:00,8.7
2024-12-31 00:00:00+00:00,8.3
